# Thông tin nhóm

1612406 - Đặng Phương Nam

1612423 - Lê Minh nghĩa

# Câu hỏi

Cho các thông tin về căn nhà:

- Đường.
- Huyện/Quận.
- Mặt tiền (met).
- Đường vào (met).
- Hướng nhà.
- Chiều dài của nhà (met).
- Chiều rộng của nhà (met).
- Chính chủ (yes/no).
- Gần trung tâm TP, dễ dàng đi lại (yes/no).
- Loại nhà (nhà cấp 4, nhà lầu hay biệt thự?).
- Hẻm (yes/no).
- Nội thất (yes/no hay giá ước lượng bao nhiêu tiền?)
- Nhà mới xây (yes/no).
- Số lượng tầng.
- Số lượng phòng ngủ.
- Số lượng toliet.

Hỏi giá trị của căn nhà là bao nhiêu tiền?

# Lợi ích khi trả lời được câu hỏi

Nhờ vào thông tin của căn nhà:

- Người bán có thể dự đoán được giá trị căn nhà mà mình muốn bán.
- Người mua có thể ước lượng được căn nhà mình muốn mua có giá cả hợp lý hay không?.
- Dự đoán được giá trị căn nhà của mình.
- ...

# Thu thập dữ liệu (parse HTML)

Dữ liệu thu thập từ trang web https://batdongsan.com.vn với bộ lọc là "Bán nhà riêng", "Khu vực TP.HCM". Thu được HTML cần parse: https://batdongsan.com.vn/ban-nha-rieng-tp-hcm

In [1]:
# Import các thư viện cần thiết
import urllib.robotparser
import json
import logging
import os.path
import time
import traceback
from datetime import datetime
from pprint import pprint

from requests_html import HTMLSession

In [2]:
# Tạo looger để ghi nhận lịch sử ở mỗi lần lấy dữ liệu
logger = logging.getLogger("crawler")
logFileName = f'{datetime.today().strftime("%Y-%m-%d_%H_%M_%S")}.log'
formatter = logging.Formatter("[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")
fhandler = logging.FileHandler(filename=logFileName)
fhandler.setLevel(logging.DEBUG)
fhandler.setFormatter(formatter)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.handlers = []
logger.addHandler(fhandler)
logger.addHandler(ch)
logger.setLevel(logging.DEBUG)


# Hàm trả về generator, generator náy trả về trang tiếp theo sau mỗi lần duyệt
def urlGenerator(baseUrl, startPage=1):
    i = startPage
    if i <= 1:
        yield baseUrl
        i = 2
    while True:
        yield f"{baseUrl}/p{i}"
        i += 1


# Hàm parse để lấy thông tin chi tiết của căn nhà từ detail Url của tin rao bán nhà.      
def parseDetailPage(session: HTMLSession, url: str):
    global logger
    logger.info(f"Processing detail url: {url}")
    
    # Lấy mã HTML của trang web
    r = session.get(url)
    
    # Dictionary lưu lại kết quả
    result = {}
    
    # Lấy phần mô tả chi tiết của bản tin rao bán căn nhà
    result["description"] = r.html.find(".pm-desc", first=True).text

    # Lấy hết các thông tin nổi bật của nhà (ví dụ: địa chỉ, mặt tiền, số phòng ngủ, số tầng, nội thất,...)
    features = {}
    featuresTable = r.html.find(".table1", first=True) # các thông tin này nằm ở class table1 trong mã HTML
    rows = featuresTable.find(".row") # lấy hết tất cả các dòng của features Table
    for row in rows:
        key = row.find(".left", first=True).text
        value = row.find(".right", first=True).text
        features[key] = value
    result["features"] = features # Thêm phần feature vào phần kết quả muốn lấy

    return result


def parseSearchResultPage(fileName: str,
                          session: HTMLSession,
                          baseUrl: str = "https://batdongsan.com.vn/ban-nha-rieng-tp-hcm",
                          startPage: int = 1):
    global logger
    if os.path.exists(fileName):
        logger.error(f"File exists {fileName}")
        return

    rp = urllib.robotparser.RobotFileParser()
    rp.set_url('https://batdongsan.com.vn/robots.txt')
    rp.read()

    try:
        with open(fileName, "w", encoding="utf8") as f:
            for url in urlGenerator(baseUrl, startPage):
                
                # Bỏ qua trang hiện tại nếu website không cho phép lấy.
                if not rp.can_fetch('*', url):
                    logger.info(f"SKIP {url}")
                    continue
                    
                logger.info(f"Processing url: {url}")
                
                # Lấy mã HTML của trang web
                r = session.get(url)
                
                # Tìm phần tử "not found" của trang kết quả tìm kiếm,
                # nếu phần tử này tồn tại tức là trang này không có kết quả nào
                # -> đã lấy hết
                notFound = r.html.find(
                    "#LeftMainContent__productSearchResult_pnlNotFound", first=True
                )
                if notFound is not None:
                    logger.info(f"{url} Not found")
                    break

                # Lấy hết các bản tin rao bán nhà
                items = r.html.find(".search-productItem")
                for item in items:
                    result = {} # Dictionary này chứa tất cả các thông tin của căn nhà cần rao bán
                    
                    # Lấy title
                    titleAnchor = item.find(".p-title a", first=True)
                    result["title"] = titleAnchor.text
                    
                    # Lấy detail Url
                    result["detailUrl"] = titleAnchor.absolute_links.pop()
                    
                    # Lấy giá nhà
                    result["price"] = item.find(".product-price", first=True).text
                    
                    # Lấy diện tích (met vuông)
                    result["area"] = item.find(".product-area", first=True).text
                    
                    # Lấy khu vực (huyện/quận)
                    result["district"] = item.find(
                        ".product-city-dist", first=True
                    ).text
                    
                    # Lấy thời gian đăng tin
                    result["uptime"] = item.find(".uptime", first=True).text
                    
                    # Kiểm tra xem có được phép lấy dữ liệu từ trang detail hay không?
                    if rp.can_fetch('*', result["detailUrl"]):
                        # Lấy thông tin mô tả chi tiết của căn nhà
                        result["detail"] = parseDetailPage(session, result["detailUrl"])
                    else:
                        logger.info(f"SKIP {result['detailUrl']}")
                    
                    # Ghi xuống file
                    json.dump(result, f, ensure_ascii=False)
                    f.write("\n")

                # sleep 2s khi lấy xong mỗi trang (chứa 20 tin đăng bán nhà)
                time.sleep(2)
                
    except Exception as e:
        logger.error(f"{e}\n{traceback.format_exc}")

In [ ]:
session = HTMLSession()

# Tạo tên file lưu lại kết quả đã parse dữ liệu từ trang web, tên file chính là thời gian tiến hành parse
resultFileName = f'data/{datetime.today().strftime("%Y-%m-%d_%H_%M_%S")}.json'

# Parse trang kết quả tìm kiếm
parseSearchResultPage(resultFileName, session)

[2019-11-30 13:21:54,431][crawler][INFO] - Processing url: https://batdongsan.com.vn/ban-nha-rieng-tp-hcm
[2019-11-30 13:21:56,750][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/ban-nha-rieng-duong-23-2-phuong-binh-tri-dong-b/can-khu-ten-lua-90m2-3-lau-co-ham-de-xe-chinh-chu-can-ban-pr23679945
[2019-11-30 13:21:57,520][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/ban-nha-rieng-duong-doan-nguyen-tuan-1-xa-hung-long-5/binh-chanh-1ty3-gan-cho-0934117173-pr22722319
[2019-11-30 13:21:58,389][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/ban-nha-rieng-duong-dong-hung-thuan-6-phuong-tan-hung-thuan-1/can-gap-5x10m2-cong-n-50m2-mot-tret-1-lung-3-lau-pr23671000
[2019-11-30 13:21:59,364][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/ban-nha-rieng-duong-lien-khu-4-5-phuong-binh-hung-hoa-b/133-16-4-5-quan-tan-tp-hcm-pr22894731
[2019-11-30 13:22:00,346][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/b

# Tiền xử lý dữ liệu

## Xóa các bản tin rao bán nhà đất bị trùng nhau

In [ ]:
# from: https://docs.python.org/3/library/itertools.html#itertools-recipes
def unique_everseen(iterable, key=None):
    "List unique elements, preserving order. Remember all elements ever seen."
    # unique_everseen('AAAABBBCCDAABBB') --> A B C D
    # unique_everseen('ABBCcAD', str.lower) --> A B C D
    seen = set()
    seen_add = seen.add
    if key is None:
        for element in filterfalse(seen.__contains__, iterable):
            seen_add(element)
            yield element
    else:
        for element in iterable:
            k = key(element)
            if k not in seen:
                seen_add(k)
                yield element


def removeDuplicates(inputFile: str, outputFile: str):
    with open(inputFile) as fIn, open(outputFile, "w") as fOut:
        for line in unique_everseen(fIn):
            fOut.write(f"{line}")


In [ ]:
dataFile = "data/2019-11-29_23_18_43.json"
noDuplicatedFile = "data/2019-11-29_23_18_43-removed-duplicate.json"

if not os.path.exists(noDuplicatedFile):
    removeDuplicates(inputFile=dataFile, outputFile=noDuplicatedFile)

## Lấy ra các thông tin cần quan tâm

In [1]:
# Import thư viện cần thiết
import json
import re

+ Đường.
+ Huyện/Quận.
+ Diện tích (met vuông)
+ Mặt tiền (met).
+ Đường vào (met).
+ Hướng nhà.
+ Hướng ban công

- Chiều dài của nhà (met).
- Chiều rộng của nhà (met).
=> Dùng Regular Expression cho dạng ở decription hay title: "3m x 6,5m", hay "ngang 5,85m x dài 5m", hay "4x19.5m" 

- Chính chủ (yes/no).
=> Search trong decription các cụm từ: "chính chủ", "pháp lý", "sổ đỏ", "sổ hồng"

- Gần trung tâm TP, dễ dàng đi lại (yes/no).
=> Search trong decription các cụm từ: "trung tâm", "sầm uất", "thuận tiện"

- Loại nhà (nhà cấp 4, nhà lầu hay biệt thự?).
=> Search "biệt thự". nếu là biệt thự thì true, còn éo phải thì false

- Hẻm (yes/no).
=> Search địa chỉ có dạng / hay không: 12/3 đường Nguyên Trãi.

- Nội thất (yes/no hay giá ước lượng bao nhiêu tiền?)
=> Search thuộc tính Nội thất có không, nếu không search trong Decription các cụm từ: "nội thất"

- Nhà mới xây (yes/no).
=> Search trong decription cụm từ: "mới xây"

+ Số lượng tầng.
+ Số lượng phòng ngủ.
+ Số lượng toliet.
+ Giá nhà.

In [ ]:
def xstr(s):
    return '' if s is None else str(s)

# Hàm chính để xử lý dữ liệu (bước lấy thông tin các cột cần quan tâm)
def preprocess(inputFile: str, outputFile: str):
    with open(inputFile) as fIn, open(outputFile, "w") as fOut:
        tab = "\t"
        headers = (
            "area",             # diện tích (m^2)
            "district",         # quận/ huyện
            "address",          # địa chỉ
            "floors",           # số lượng tầng 
            "bedrooms",         # số lượng phòng ngủ
            "toilets",          # số lượng toitlet
            "front",            # mặt tiền hay chiều ngang của nhà (m)
            "entrance",         # dường vào nhà (m)
            "house_aspect",     # hướng của nhà
            "balcony_aspect",   # hướng của ban công
            "interior",         # nội thất (có hay không)
            "near_center",      # gần trung tâm (ở các quận 1, 3, 4 và Bình Thạnh -> có hay không)
            "owner",            # chính chủ, sổ đỏ (có hay không)
            "alley",            # hẻm (có hay không)
            "villa",            # biệt thự (có hay không)
            "new",              # nhà mới (có hay không)
            "price",            # giá nhà (triệu đồng)
        )
        
        # Ghi các tên cột vào file output, mỗi cột ngăn cách bởi "\t"
        fOut.write(f"{tab.join(headers)}\n")
        
        for i, line in enumerate(fIn): # Mở file input để tiến hành lấy dữ liệu cần thiết    
            # Lấy từng dòng
            row = json.loads(line)
            
            # Lấy diện tích nhà
            if row["area"] == "Không xác định":
                area = None
            else:
                area = float(row["area"][:-3])

            # Lấy giá nhà
            if row["price"].endswith(" tỷ"):
                price = float(row["price"][:-3]) * 1000
            elif row["price"].endswith(" triệu"):
                price = float(row["price"][:-6])
            elif row["price"].endswith(" triệu/m²"):
                if area is not None:
                    price = float(row["price"][:-9]) * area
                else:
                    price = None
            else:
                price = None

            if price is None:
                continue
                
            # Lấy tên quận/huyện
            district = row["district"]
            detail = row["detail"]
            features = detail["features"]

            # Lấy địa chỉ nhà
            address = features.get("Địa chỉ")
            if address is not None:
                address = address.replace(district, "").strip(" \t\r\n,.")
                if len(address) == 0:
                    address = None
            district = district.replace("Hồ Chí Minh", "").strip(" \t\r\n,.")

            # Lấy số lượng tầng của nhà
            floors = features.get("Số tầng")
            if floors is not None:
                floors = int(floors.replace(" (tầng)", ""))

            # Lấy số lượng phòng ngủ của nhà
            bedrooms = features.get("Số phòng ngủ")
            if bedrooms is not None:
                bedrooms = int(bedrooms.replace(" (phòng)", ""))

            # Lấy số lượng toilet của nhà
            toilets = features.get("Số toilet")
            if toilets is not None:
                toilets = int(toilets)

            # Lấy kích thước mặt tiền của nhà
            front = features.get("Mặt tiền")
            if front is not None:
                front = float(front.replace(" (m)", "").replace(",", "."))

            # Lấy kích thước đường vào nhà
            entrance = features.get("Đường vào")
            if entrance is not None:
                entrance = float(entrance.replace(" (m)", "").replace(",", "."))

            # Lấy hướng nhà
            houseAspect = features.get("Hướng nhà")
            
            # Lấy hướng ban công của nhà
            balconyAspect = features.get("Hướng ban công")
            if balconyAspect is not None:
                if balconyAspect == "KXĐ":
                    balconyAspect = None

            # Phần mô tả chi tiết về căn nhà cần bán
            description = detail["description"].lower()

            # Lấy thông tin nội thất: có/không
            interiorStr = features.get("Nội thất")
            if interiorStr is None:
                interior = None
            else:
                interiorStr = interiorStr.lower()
                noStartStrs = ["ko", "khong", "không", "k ", "kxđ"]
                # fmt: off
                if interiorStr == "k" or any(interiorStr.startswith(s) for s in noStartStrs):
                    interior = False
                else:
                    interior = True
                # fmt: on

            # match = re.match(
            #     r"([0-9]*[.,]?[0-9]+)\s*m?\s*[*x]\s*([0-9]*[.,]?[0-9]+)", description,
            # )
            # if match:
            #     width, length = match.group(1, 2)
            # else:
            #     pass

            # Lấy thông tin gần trung tâm: có/không
            nearCenter = district in ["Quận 1", "Quận 3", "Quận 4", "Bình Thạnh"]
            
            # Lấy thông tin chính chủ, sổ đỏ: có/không
            owner = any(
                searchStr in description
                for searchStr in ["sổ đỏ", "sổ hồng", "chính chủ", "pháp lý"]
            )

            # Lấy thông tin hẻm: có/không
            alley = False
            if address is not None:
                alley = "/" in address
            if not alley:
                alley = "hẻm" in description

            # Lấy thông tin biệt thự: có/không
            villa = "biệt thự" in description
            new = any(
                searchStr in description
                for searchStr in ["nhà mới", "mới xây", "mới 100"]
            )

            # Tạo một dòng dữ liệu mới từ các thông tin trên
            row = (
                xstr(area),
                xstr(district),
                xstr(address),
                xstr(floors),
                xstr(bedrooms),
                xstr(toilets),
                xstr(front),
                xstr(entrance),
                xstr(houseAspect),
                xstr(balconyAspect),
                xstr(interior),
                xstr(nearCenter),
                xstr(owner),
                xstr(alley),
                xstr(villa),
                xstr(new),
                xstr(price),
            )
            
            # Ghi xuống file output
            fOut.write(f"{tab.join(row)}\n")

In [ ]:
# Chạy hàm xử lý dữ liệu
dataFile = "data/2019-11-29_23_18_43-removed-duplicate.json"
outputFile = "data/2019-11-29_23_18_43.csv"

preprocess(inputFile=dataFile, outputFile=outputFile)

## Xem thông tin về dữ liệu

In [2]:
import pandas as pd

In [3]:
data_df =  pd.read_csv("data/2019-11-29_23_18_43.csv", sep='\t')
data_df.head(10)

,area,district,address,floors,bedrooms,toilets,front,entrance,house_aspect,balcony_aspect,interior,near_center,owner,alley,villa,new,price
0,80.0,Bình Chánh,"Đường Đoàn Nguyễn Tuân, Xã Hưng Long",2.0,3.0,2.0,4.00,6.0,Đông,NaN,NaN,0,1,0,0,0,1200.0
1,50.0,Quận 12,"Đường Đông Hưng Thuận 6, Phường Tân Hưng Thuận",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,4390.0
2,72.0,Bình Tân,"133/16, Đường Liên khu 4-5, Phường Bình Hưng H...",1.0,2.0,NaN,4.00,6.0,Đông,NaN,NaN,0,1,1,0,1,3600.0
3,65.0,Thủ Đức,"Đường 8, Phường Hiệp Bình Chánh",4.0,4.0,5.0,NaN,6.0,NaN,NaN,1.0,0,1,1,0,1,6199.0
4,58.0,Gò Vấp,"Đường Phạm Văn Chiêu, Phường 9",5.0,5.0,6.0,5.30,NaN,Tây-Nam,NaN,1.0,0,0,1,1,0,6800.0
5,56.0,Gò Vấp,"Đường 4, Phường 8",4.0,4.0,5.0,4.00,NaN,Đông-Bắc,NaN,1.0,0,1,1,0,1,5600.0
6,40.0,Bình Chánh,NaN,2.0,4.0,2.0,4.00,NaN,NaN,NaN,1.0,0,1,0,0,1,680.0
7,123.0,Quận 9,"Số 74/7/9, Đường 8, Phường Trường Thạnh",3.0,4.0,3.0,7.00,6.0,Tây-Bắc,Tây-Bắc,1.0,0,1,1,0,0,3500.0
8,30.0,Quận 1,"115/34, Đường Trần Đình Xu, Phường Nguyễn Cư T...",4.0,4.0,4.0,3.00,NaN,NaN,NaN,NaN,1,1,1,0,1,4750.0
9,80.0,Thủ Đức,"Đường 16, Phường Hiệp Bình Chánh",3.0,3.0,4.0,6.36,NaN,NaN,NaN,1.0,0,1,0,0,1,6470.0


In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19443 entries, 0 to 19442
Data columns (total 17 columns):
area              18261 non-null float64
district          19443 non-null object
address           19073 non-null object
floors            12277 non-null float64
bedrooms          8828 non-null float64
toilets           7907 non-null float64
front             8257 non-null float64
entrance          10182 non-null float64
house_aspect      3511 non-null object
balcony_aspect    1604 non-null object
interior          3457 non-null float64
near_center       19443 non-null int64
owner             19443 non-null int64
alley             19443 non-null int64
villa             19443 non-null int64
new               19443 non-null int64
price             19443 non-null float64
dtypes: float64(8), int64(5), object(4)
memory usage: 2.5+ MB


In [13]:
data_df.shape

(19443, 11)